In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

피쳐를 수정,추가 하다 원본이 없어져서 다시 시작한 경험 후에는
원본을 카피

In [2]:
org_tr = pd.read_csv('/Users/kyhh/Desktop/encore21/train.csv')
org_te = pd.read_csv('/Users/kyhh/Desktop/encore21/test.csv')

In [3]:
tr = org_tr.copy()
te = org_te.copy()

In [4]:
tr.shape , te.shape

((916, 12), (393, 11))

# EDA

In [5]:
tr

,passengerid,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked
0,0,0,2,"Wheeler, Mr. Edwin Frederick""""",male,NaN,0,0,SC/PARIS 2159,12.8750,NaN,S
1,1,0,3,"Henry, Miss. Delia",female,NaN,0,0,382649,7.7500,NaN,Q
2,2,1,1,"Hays, Mrs. Charles Melville (Clara Jennings Gr...",female,52.0,1,1,12749,93.5000,B69,S
3,3,1,3,"Andersson, Mr. August Edvard (""Wennerstrom"")",male,27.0,0,0,350043,7.7958,NaN,S
4,4,0,2,"Hold, Mr. Stephen",male,44.0,1,0,26707,26.0000,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
911,911,0,3,"Thomas, Mr. Charles P",male,NaN,1,0,2621,6.4375,NaN,C
912,912,0,3,"Cacic, Mr. Jego Grga",male,18.0,0,0,315091,8.6625,NaN,S
913,913,0,2,"Pengelly, Mr. Frederick William",male,19.0,0,0,28665,10.5000,NaN,S
914,914,1,3,"Mullens, Miss. Katherine ""Katie""",female,NaN,0,0,35852,7.7333,NaN,Q


In [6]:
tr['embarked']=tr['embarked'].fillna('S')
te['embarked']=te['embarked'].fillna('S')

In [7]:
tr.drop(['cabin'],axis =1 ,inplace=True)
te.drop(['cabin'],axis =1 ,inplace=True)

In [8]:
tr['age'] = tr['age'].fillna(29)
te['age'] = te['age'].fillna(29)

In [9]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 916 entries, 0 to 915
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   passengerid  916 non-null    int64  
 1   survived     916 non-null    int64  
 2   pclass       916 non-null    int64  
 3   name         916 non-null    object 
 4   gender       916 non-null    object 
 5   age          916 non-null    float64
 6   sibsp        916 non-null    int64  
 7   parch        916 non-null    int64  
 8   ticket       916 non-null    object 
 9   fare         916 non-null    float64
 10  embarked     916 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 78.8+ KB


In [10]:
features = tr.drop(['passengerid','survived'],axis=1)
target = tr['survived']

In [11]:
features['embarked'] = features['embarked'].map({'S':1,'Q':2,'C':3})
te['embarked'] = te['embarked'].map({'S':1,'Q':2,'C':3})

In [12]:
features['gender'] = features['gender'].map({'male':1,'female':2})
te['gender'] = te['gender'].map({'male':1,'female':2})

In [13]:
te['fare'] = te['fare'].fillna(35)

In [14]:
features['fare'] = features['fare'].astype(int)
te['fare'] = te['fare'].astype(int)

In [15]:
features['age'] = features['age'].astype(int)
te['age'] = te['age'].astype(int)

In [16]:
def add_sub_age(age):
    result = None
    try:
        if age <=20:
            result = 1
        elif age <=30:
            result = 2
        elif age <=50:
            result = 3
        else:
            result = 4
    except:
        pass
    return result 

features['sub_age'] = features['age'].map(lambda x: add_sub_age(x))
te['sub_age'] = te['age'].map(lambda x: add_sub_age(x))

features[['sub_age', 'age']].head()

,sub_age,age
0,2,29
1,2,29
2,4,52
3,2,27
4,3,44


In [17]:
features = features.drop(['age'],axis=1)
te = te.drop(['age'],axis=1)

In [18]:
def add_sub_fare(fare):
    result = None
    try:
        if fare <=50:
            result = 4
        elif fare <=100:
            result = 3
        elif fare <=300:
            result = 2
        else:
            result = 1
    except:
        pass
    return result
    
features['sub_fare'] = features['fare'].map(lambda x: add_sub_fare(x))
te['sub_fare'] = te['fare'].map(lambda x: add_sub_fare(x))
features[['fare','sub_fare']].head()

,fare,sub_fare
0,12,4
1,7,4
2,93,3
3,7,4
4,26,4


In [19]:
dict_designation = {
    'Mr.': '남성',
    'Master.': '남성',
    'Sir.': '남성',
    'Miss.': '미혼 여성',
    'Mrs.': '기혼 여성',
    'Ms.': '미혼/기혼 여성',
    'Lady.': '숙녀',
    'Mlle.': '아가씨',
    # 직업
    'Dr.': '의사',
    'Rev.': '목사',
    'Major.': '계급',
    'Don.': '교수',
    'Col.': '군인',
    'Capt.': '군인',
    # 귀족
    'Mme.': '영부인',
    'Countess.': '백작부인',
    'Jonkheer.': '귀족'
}

def add_designation(name): # 호칭 함수
    designation = "unknown"
    for key in dict_designation.keys():
        if key in name:
            designation = key
            break
    return designation

features['designation'] = features['name'].map(lambda x: add_designation(x))
te['designation'] = te['name'].map(lambda x: add_designation(x))

features.shape, te.shape 

((916, 11), (393, 12))

In [20]:
features['last_name'] = features['name'].map(lambda x: x.split(',')[0])
te['last_name'] = te['name'].map(lambda x: x.split(',')[0])

features.shape, te.shape 

((916, 12), (393, 13))

In [21]:
def add_ticket_number(ticket):
    try:
        ticket_split = ticket.split(' ')
        return int(ticket_split[-1])
    except:
        return 0 # ticket이 LINE인 경우

features['ticket_number'] = features['ticket'].map(lambda x: add_ticket_number(x)).astype("int32")
te['ticket_number'] = te['ticket'].map(lambda x: add_ticket_number(x)).astype("int32")

features.shape, te.shape 

((916, 13), (393, 14))

In [22]:
# features = features.drop(['name','ticket'],axis=1)
# te = te.drop(['name','ticket'],axis=1)

In [23]:
features = features.drop(['fare'],axis=1)
te = te.drop(['fare'],axis=1)


In [24]:
features['famliy'] = features['sibsp'] + features['parch'] +1
te['famliy'] = te['sibsp'] + te['parch'] +1

In [25]:
def add_sub_sibsp(sibsp):
    result = None
    try:
        if sibsp <=1 :
            result = 0
        elif sibsp <=3:
            result = 1
        elif sibsp <=5:
            result = 2
        else:
            result = 3
    except:
        pass
    return result

features['sub_sibsp'] = features['sibsp'].map(lambda x: add_sub_sibsp(x))
te['sub_sibsp'] = te['sibsp'].map(lambda x: add_sub_sibsp(x))

features[['sibsp','sub_sibsp']].head()

,sibsp,sub_sibsp
0,0,0
1,0,0
2,1,0
3,0,0
4,1,0


In [26]:
def add_sub_parch(parch):
    result = None
    try:
        if parch <= 1 :
            result = 0
        elif parch <= 4:
            result = 1
        else:
            result = 2
    except:
        pass
    return result

features['sub_parch'] = features['parch'].map(lambda x : add_sub_parch(x))
te['sub_parch'] = te['parch'].map(lambda x : add_sub_parch(x))

features[['parch','sub_parch']]

,parch,sub_parch
0,0,0
1,0,0
2,1,0
3,0,0
4,0,0
...,...,...
911,0,0
912,0,0
913,0,0
914,0,0


In [27]:
features.shape , te.shape

((916, 15), (393, 16))

In [28]:
features.drop(['parch','sibsp'],axis=1,inplace=True)
te.drop(['parch','sibsp'],axis=1,inplace=True)

In [29]:
te.set_index(['passengerid'], inplace=True)
print(f'{te.shape}')
te.head()

(393, 13)


,pclass,name,gender,ticket,embarked,sub_age,sub_fare,designation,last_name,ticket_number,famliy,sub_sibsp,sub_parch
passengerid,,,,,,,,,,,,,
916,3,"McGowan, Miss. Anna ""Annie""",2,330923,2,1,4,Miss.,McGowan,330923,1,0,0
917,2,"Pinsky, Mrs. (Rosa)",2,234604,1,3,4,Mrs.,Pinsky,234604,1,0,0
918,3,"McCarthy, Miss. Catherine Katie""""",2,383123,2,2,4,Miss.,McCarthy,383123,1,0,0
919,3,"Franklin, Mr. Charles (Charles Fardon)",1,SOTON/O.Q. 3101314,1,2,4,Mr.,Franklin,3101314,1,0,0
920,1,"Wick, Mrs. George Dennick (Mary Hitchcock)",2,36928,1,3,2,Mrs.,Wick,36928,3,0,0


In [30]:
features.shape , te.shape

((916, 13), (393, 13))

In [31]:
features

,pclass,name,gender,ticket,embarked,sub_age,sub_fare,designation,last_name,ticket_number,famliy,sub_sibsp,sub_parch
0,2,"Wheeler, Mr. Edwin Frederick""""",1,SC/PARIS 2159,1,2,4,Mr.,Wheeler,2159,1,0,0
1,3,"Henry, Miss. Delia",2,382649,2,2,4,Miss.,Henry,382649,1,0,0
2,1,"Hays, Mrs. Charles Melville (Clara Jennings Gr...",2,12749,1,4,3,Mrs.,Hays,12749,3,0,0
3,3,"Andersson, Mr. August Edvard (""Wennerstrom"")",1,350043,1,2,4,Mr.,Andersson,350043,1,0,0
4,2,"Hold, Mr. Stephen",1,26707,1,3,4,Mr.,Hold,26707,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,3,"Thomas, Mr. Charles P",1,2621,3,2,4,Mr.,Thomas,2621,2,0,0
912,3,"Cacic, Mr. Jego Grga",1,315091,1,1,4,Mr.,Cacic,315091,1,0,0
913,2,"Pengelly, Mr. Frederick William",1,28665,1,1,4,Mr.,Pengelly,28665,1,0,0
914,3,"Mullens, Miss. Katherine ""Katie""",2,35852,2,2,4,Miss.,Mullens,35852,1,0,0


In [32]:
features.drop(['name','ticket'],axis=1,inplace=True)
te.drop(['name','ticket'],axis=1,inplace=True)

In [33]:
features.drop(['last_name'],axis=1,inplace=True)
te.drop(['last_name'],axis=1,inplace=True)

In [34]:
features['designation'].nunique()

15

In [35]:
def assign_sub_designation(designation):
    if designation in ['Countess.', 'Sir', 'Don.', 'Jonkheer.', 'Capt.']:
        return 1
    elif designation in ['Col.', 'Major.', 'Dr.', 'Rev.']:
        return 2
    elif designation in ['Mrs.', 'Mr.', 'Miss.', 'Ms.', 'Mlle.']:
        return 3
    else:
        return 0  # 해당되는 것이 없으면 0을 반환

features['sub_designation'] = features['designation'].apply(lambda x: assign_sub_designation(x))
te['sub_designation'] = te['designation'].apply(lambda x: assign_sub_designation(x))


features[['designation','sub_designation']].head()

,designation,sub_designation
0,Mr.,3
1,Miss.,3
2,Mrs.,3
3,Mr.,3
4,Mr.,3


In [36]:
features['sub_designation'].nunique()

4

In [37]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 916 entries, 0 to 915
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   pclass           916 non-null    int64 
 1   gender           916 non-null    int64 
 2   embarked         916 non-null    int64 
 3   sub_age          916 non-null    int64 
 4   sub_fare         916 non-null    int64 
 5   designation      916 non-null    object
 6   ticket_number    916 non-null    int32 
 7   famliy           916 non-null    int64 
 8   sub_sibsp        916 non-null    int64 
 9   sub_parch        916 non-null    int64 
 10  sub_designation  916 non-null    int64 
dtypes: int32(1), int64(9), object(1)
memory usage: 75.3+ KB


In [38]:
features.drop(['designation'],axis=1,inplace=True)
te.drop(['designation'],axis=1,inplace=True)

In [39]:
features.shape , te.shape

((916, 10), (393, 10))

In [40]:
features

,pclass,gender,embarked,sub_age,sub_fare,ticket_number,famliy,sub_sibsp,sub_parch,sub_designation
0,2,1,1,2,4,2159,1,0,0,3
1,3,2,2,2,4,382649,1,0,0,3
2,1,2,1,4,3,12749,3,0,0,3
3,3,1,1,2,4,350043,1,0,0,3
4,2,1,1,3,4,26707,2,0,0,3
...,...,...,...,...,...,...,...,...,...,...
911,3,1,3,2,4,2621,2,0,0,3
912,3,1,1,1,4,315091,1,0,0,3
913,2,1,1,1,4,28665,1,0,0,3
914,3,2,2,2,4,35852,1,0,0,3


# OneHotEncoder 소용없음

In [41]:
# from sklearn.preprocessing import OneHotEncoder

# enc_cols = ['pclass' , 'gender' , 'embarked']
# enc = OneHotEncoder()

# tmp_feat = pd.DataFrame(enc.fit_transform(features[enc_cols]).toarray(),columns=enc.get_feature_names_out())
# features = pd.concat([features.reset_index(drop=True),tmp_feat.reset_index(drop=True)],axis=1)

# tmp_te = pd.DataFrame(enc.fit_transform(te[enc_cols]).toarray(),columns=enc.get_feature_names_out())
# te = pd.concat([te.reset_index(drop=True),tmp_te.reset_index(drop=True)],axis=1)

In [42]:
# features.drop(['pclass','gender','embarked'],axis=1,inplace=True)
# te.drop(['pclass','gender','embarked'],axis=1,inplace=True)

# 모델학습

In [43]:
from sklearn.model_selection import train_test_split

In [44]:
X_tr,X_te,y_tr,y_te = train_test_split(features,target,random_state=1,test_size=0.2)

X_tr.shape , X_te.shape, te.shape

((732, 10), (184, 10), (393, 10))

# RandomForest

In [45]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200,random_state=1,max_depth=5).fit(X_tr,y_tr)

In [46]:
print(f'트레인 : {rf.score(X_tr,y_tr)} / 테스트: {rf.score(X_te,y_te)}')

트레인 : 0.8852459016393442 / 테스트: 0.8641304347826086


In [47]:
hp = {
    "random_state" : 0,
    "max_features" : "sqrt", # None 을 줄경우 전체피쳐 사용 
    "n_estimators" : 100, # 트리개수 
    "max_depth" : 10,
    "min_samples_split" : 10,
    "min_samples_leaf" : 3,
}

rf = RandomForestClassifier(**hp).fit(X_tr, y_tr)
print(f'훈련용 평가지표: {rf.score(X_tr, y_tr)} / 테스트용 평가지표: {rf.score(X_te, y_te)}')

훈련용 평가지표: 0.8934426229508197 / 테스트용 평가지표: 0.8641304347826086


# Gradient

In [48]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=200,random_state=1,max_depth=5).fit(X_tr,y_tr)
print(f'훈련용 평가지표: {gb.score(X_tr, y_tr)} / 테스트용 평가지표: {gb.score(X_te, y_te)}')

훈련용 평가지표: 0.9959016393442623 / 테스트용 평가지표: 0.8586956521739131


# Lightgbm

In [49]:
from lightgbm import LGBMRegressor, LGBMClassifier
lgbm = LGBMClassifier(random_state=1)
lgbm.fit(X_tr, y_tr)
preds = lgbm.predict_proba(te)
lgbm.score(X_tr,y_tr) , lgbm.score(X_te,y_te)

(0.9549180327868853, 0.8478260869565217)

In [50]:
from lightgbm import LGBMRegressor, LGBMClassifier

hp = {  
    'n_estimators': 1193,
    'num_leaves': 228,
    'learning_rate': 0.18860436401232483,
    'min_child_samples': 8,
    'subsample': 0.656886992827764,
    'max_depth': 6,
    'colsample_bytree': 0.21329750566563033,
    'reg_alpha': 0.2651435044914211,
    'reg_lambda': 0.14198788151696745,
    'min_split_gain': 0.9985404497459053
    }

lgbm = LGBMClassifier(**hp).fit(X_tr, y_tr)
print(f'훈련용 평가지표: {lgbm.score(X_tr, y_tr)} / 테스트용 평가지표: {lgbm.score(X_te, y_te)}')

훈련용 평가지표: 0.8825136612021858 / 테스트용 평가지표: 0.8586956521739131


# CSV 파일 뽑기

In [51]:
# submission = pd.read_csv('/Users/kyhh/Desktop/encore21/submission.csv')
# test_pred = lgbm.predict_proba(te)[:,1]
# submission['survived'] = test_pred
# submission.to_csv('submission_lgbm_2.csv', header=True, index=False)

# Lightgbm 파라미터

In [52]:
# pip install lightgbm

In [53]:
# pip install optuna

In [54]:
# import lightgbm as lgb
# import optuna
# from sklearn.model_selection import train_test_split
# import sklearn

# def objective(trial):
#     dtrain = lgb.Dataset(X_tr, label=y_tr)
#     param = {
#         'n_estimators' : trial.suggest_int('n_estimators',100,3000),
#         'objective': 'binary', 
#         'verbose': -1,
#         'metric': 'binary_logloss',
#         'boosting_type': 'gbdt',
#         'num_leaves': trial.suggest_int('num_leaves', 2, 256),
#         'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1.0),
#         'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
#         'subsample': trial.suggest_float('subsample', 0.1, 1.0),
#         'max_depth': trial.suggest_int('max_depth', 3, 20),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
#         'reg_alpha': trial.suggest_float('reg_alpha', 1e-16, 1.0),
#         'reg_lambda': trial.suggest_float('reg_lambda', 1e-16, 1.0),
#         'min_split_gain': trial.suggest_float('min_split_gain', 0, 1),
#     }
#     gbm = lgb.train(param, dtrain)
#     preds = gbm.predict(X_te)
#     pred_labels = np.rint(preds)
#     accuracy = sklearn.metrics.accuracy_score(y_te, pred_labels)
#     return accuracy

# X_tr, X_te, y_tr, y_te = train_test_split(features, target, test_size=0.2, random_state=42)

# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=100)

# print("Number of finished trials: ", len(study.trials))
# print("Best trial:")
# trial = study.best_trial
# print("  Value: {}".format(trial.value))
# print("  Params: ")
# for key, value in trial.params.items():
#     print("    {}: {}".format(key, value))